### Defining DataLoader and downloading Dataset

In [5]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define a transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
dataset = datasets.MNIST(root='/root/data', train=True, download=True, transform=transform)

# Create DataLoader with multiple workers
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 130090776.45it/s]


Extracting /root/data/MNIST/raw/train-images-idx3-ubyte.gz to /root/data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 8446220.46it/s]

Extracting /root/data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 28197513.71it/s]

Extracting /root/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2539054.88it/s]

Extracting /root/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/data/MNIST/raw



### Iterating through dataloader

In [ ]:

# Iterate through the DataLoader
for batch_idx, (inputs, targets) in enumerate(dataloader):
    print(f"Batch {batch_idx+1}")
    print(f"Inputs shape: {inputs.shape}")
    print(f"Targets shape: {targets.shape}")
    # Here you can add your training code, e.g., forward pass, loss calculation, etc.
    # For demonstration, we'll just print the shapes.


### Creating DataLoader for Custom Dataset


In [9]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Define the custom dataset class
class RandomDataset(Dataset):
    def __init__(self, num_samples, image_shape):
        self.num_samples = num_samples
        self.image_shape = image_shape

    def __len__(self):
        return self.num_samples #returns the length of the dataset

    def __getitem__(self, idx):
        # Generate random image
        image = torch.randn(*self.image_shape) #generate random image

        # Generate random label
        label = torch.randint(0, 10, ()) #generate random label

        return image, label

# Define the dataset parameters
num_samples = 1000
image_shape = (1, 28, 28)  # (channels, height, width)

# Load your training data
train_dataset   = RandomDataset(num_samples, image_shape) # Generate random dataset Your training dataset

In [10]:
# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
print(len(train_loader))

16


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your neural network architecture
class NeuralNet(nn.Module): #inheretence concept
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU() #whether you will use this AF many times or not , Define it once
        
    def forward(self, x):
        x = self.flatten(x) # 28x28 --> 784 
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x) 
        return x

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Instantiate the neural network
model = NeuralNet()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss() #Classification or regression? 
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the neural network
for epoch in range(10): # 10  Epochs
    running_loss = 0.0
    for inputs, labels in train_loader: #looping the batches
        
        # ToDo: Enable gradient tracking
        
        # Zero the gradients
        optimizer.zero_grad()  #ensures that the gradients are cleared or reset to zero before the next forward-backward pass.

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward() #calculate the gradients
        #ToDo: Print Gradients
        optimizer.step() #update our weights of each Batch

        running_loss += loss.item() #accumulating losses

    print(f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader)}") # Batch loss=Total losses/number of batches
    #ToDo: Save your model will be only done if your loss < the minimum calculated loss (model.pt)


Epoch 1: Loss = 2.3259033411741257
Epoch 2: Loss = 2.325193867087364
Epoch 3: Loss = 2.319349765777588
Epoch 4: Loss = 2.3052792251110077
Epoch 5: Loss = 2.3253238201141357
Epoch 6: Loss = 2.315476641058922
Epoch 7: Loss = 2.3255495578050613
Epoch 8: Loss = 2.3312793970108032
Epoch 9: Loss = 2.315801292657852
Epoch 10: Loss = 2.320280820131302


In [16]:
model.fc1.weight.grad

tensor([[-0.0018,  0.0014, -0.0014,  ..., -0.0002, -0.0027, -0.0064],
        [-0.0020, -0.0068,  0.0039,  ..., -0.0036,  0.0032, -0.0042],
        [-0.0011,  0.0022, -0.0007,  ...,  0.0062,  0.0013,  0.0038],
        ...,
        [-0.0069,  0.0042,  0.0015,  ..., -0.0032, -0.0077, -0.0015],
        [ 0.0015, -0.0043, -0.0021,  ..., -0.0008, -0.0016, -0.0038],
        [ 0.0019,  0.0003, -0.0032,  ..., -0.0008,  0.0018,  0.0005]])

In [13]:
model.fc1.bias.grad

tensor([-3.5767e-03,  4.2845e-03,  3.4003e-03,  2.5443e-03, -3.1816e-03,
        -2.9246e-03,  1.4145e-03, -3.0543e-03, -3.3440e-03, -3.3123e-03,
         3.8662e-03, -1.6881e-04, -6.0021e-04,  4.4093e-03,  2.5267e-03,
        -1.3221e-03, -7.2000e-03, -2.2067e-03, -2.6351e-03, -5.7326e-03,
        -4.1427e-04, -7.1342e-03, -5.6208e-04, -5.6288e-03, -2.8622e-03,
         1.2146e-04,  1.7808e-03,  7.7193e-04,  4.0079e-03,  1.3993e-03,
        -3.6245e-03,  1.0745e-03,  4.4239e-03, -2.0509e-03, -3.7758e-03,
         6.2648e-03, -6.0815e-03,  3.7772e-04, -2.7438e-03, -3.8099e-03,
         4.3043e-03,  1.7479e-03,  6.4282e-04, -7.2688e-03,  1.6816e-03,
        -4.1388e-03, -2.3411e-03,  7.6347e-04, -8.6439e-04,  1.4495e-03,
        -8.5094e-03,  1.3149e-03, -3.2493e-04, -1.4585e-03,  3.6696e-03,
        -6.7622e-04, -2.7654e-03, -3.9413e-03,  3.4215e-03,  4.4204e-03,
         1.6653e-03,  3.3769e-03, -3.0891e-03,  4.8049e-03, -2.2066e-03,
        -4.0369e-03, -2.6313e-05,  3.4641e-03,  4.4